# Global Stability Analysis for Distributed Law

In [29]:
import numpy as np
from math import sin, cos, atan2
import torch
import sympy as sp
from scipy import integrate
import tqdm
import multiprocessing
from tqdm.contrib.concurrent import process_map

### Simulation function

In [30]:

sample_count = 100000

def foot(x, t, l, k1, k2):
    a = -(x[4] - k1)
    b = -(x[5] - k2)
    t1 = x[6]
    t2 = x[7]
    t3 = x[8]
    p1 = sin(t1)
    p2 = cos(t1)
    p3 = sin(t2)
    p4 = cos(t2)
    p5 = sin(t3)
    p6 = cos(t3)
    p7 = sin(t1 + t2)
    p8 = cos(t1 + t2)
    p9 = sin(t1 + t2 + t3)
    p10 = cos(t1 + t2 + t3)
    p11 = sin(t2 + t3)
    p12 = cos(t2 + t3)
    x_prime = b * p9 + a * p10
    y_prime = b * p10 - a * p9
    a = x_prime
    b = y_prime
    x1dot = -l * p1 * (atan2((l * p3 + l * p11 + a * p11 + b * p12), (l + l * p4 + l * p12 + a * p12 - b * p11)) -
                      atan2((l * p3 + l * p11), (l + l * p4 + l * p12)))
    x2dot = l * p2 * (atan2((l * p3 + l * p11 + a * p11 + b * p12), (l + l * p4 + l * p12 + a * p12 - b * p11)) -
                     atan2((l * p3 + l * p11), (l + l * p4 + l * p12)))
    x3dot = (-l * p1 - l * p7) * (atan2((l * p3 + l * p11 + a * p11 + b * p12), (l + l * p4 + l * p12 + a * p12 - b * p11)) -
                                 atan2((l * p3 + l * p11), (l + l * p4 + l * p12))) - l * p7 * (atan2((l * p5 + a * p5 + b * p6), (l + l * p6 + a * p6 - b * p5)) -
                                                                                                atan2((l * p5), (l + l * p6)))
    x4dot = (l * p2 + l * p8) * (atan2((l * p3 + l * p11 + a * p11 + b * p12), (l + l * p4 + l * p12 + a * p12 - b * p11)) -
                                atan2((l * p3 + l * p11), (l + l * p4 + l * p12))) + l * p8 * (atan2((l * p5 + a * p5 + b * p6), (l + l * p6 + a * p6 - b * p5)) -
                                                                                               atan2((l * p5), (l + l * p6)))
    x5dot = (-l * p1 - l * p7 - l * p9) * (atan2((l * p3 + l * p11 + a * p11 + b * p12), (l + l * p4 + l * p12 + a * p12 - b * p11)) -
                                          atan2((l * p3 + l * p11), (l + l * p4 + l * p12))) + (-l * p7 - l * p9) * (atan2((l * p5 + a * p5 + b * p6), (l + l * p6 + a * p6 - b * p5)) -
                                                                                                                 atan2((l * p5), (l + l * p6))) - l * p9 * atan2(b, (l + a))
    x6dot = (l * p2 + l * p8 + l * p10) * (atan2((l * p3 + l * p11 + a * p11 + b * p12), (l + l * p4 + l * p12 + a * p12 - b * p11)) -
                                          atan2((l * p3 + l * p11), (l + l * p4 + l * p12))) + (l * p8 + l * p10) * (atan2((l * p5 + a * p5 + b * p6), (l + l * p6 + a * p6 - b * p5)) -
                                                                                                                 atan2((l * p5), (l + l * p6))) + l * p10 * atan2(b, (l + a))
    t1dot = (atan2((l * p3 + l * p11 + a * p11 + b * p12), (l + l * p4 + l * p12 + a * p12 - b * p11)) -
             atan2((l * p3 + l * p11), (l + l * p4 + l * p12)))
    t2dot = (atan2((l * p5 + a * p5 + b * p6), (l + l * p6 + a * p6 - b * p5)) -
             atan2((l * p5), (l + l * p6)))
    t3dot = atan2(b, (l + a))
    dXdt = np.array([x1dot, x2dot, x3dot, x4dot, x5dot, x6dot, t1dot, t2dot, t3dot])
    return dXdt



### System Variable Definitions

In [31]:

# Define all the system variables
delta_1, delta_2, delta_3, phi_1, phi_2, phi_3, alpha, R, l, D_11, D_12, D_13 = sp.symbols('delta_1 delta_2 delta_3 phi_1 phi_2 phi_3 alpha R l D_11 D_12 D_13')

delta_1 = alpha - sp.atan2((sp.sin(phi_1) + sp.sin(phi_1 + phi_2) + sp.sin(phi_1 + phi_2 + phi_3)), (sp.cos(phi_1) + sp.cos(phi_1 + phi_2) + sp.cos(phi_1 + phi_2 + phi_3)))
delta_2 = sp.atan2((R * sp.sin(alpha) - l * sp.sin(phi_1)), (R * sp.cos(alpha) - l * sp.cos(phi_1))) - sp.atan2((sp.sin(phi_1 + phi_2) + sp.sin(phi_1 + phi_2 + phi_3)), (sp.cos(phi_1 + phi_2) + sp.cos(phi_1 + phi_2 + phi_3)))
delta_3 = sp.atan2((R * sp.sin(alpha) - 2 * l * sp.sin(phi_1 + phi_2 / 2) * sp.cos(phi_2 / 2)), (R * sp.cos(alpha) - 2 * l * sp.cos(phi_1 + phi_2 / 2) * sp.cos(phi_2 / 2))) - (phi_1 + phi_2 + phi_3)


c_1 = sp.cos(phi_1)
c_12 = sp.cos(phi_1 + phi_2)
c_123 = sp.cos(phi_1 + phi_2 + phi_3)
c_3 = sp.cos(phi_3)
c_23 = sp.cos(phi_2 + phi_3)
c_2 = sp.cos(phi_2)

s_1 = sp.sin(phi_1)
s_12 = sp.sin(phi_1 + phi_2)
s_123 = sp.sin(phi_1 + phi_2 + phi_3)
s_3 = sp.sin(phi_3)
s_23 = sp.sin(phi_2 + phi_3)
s_2 = sp.sin(phi_2)

y_1 = l * s_1
y_2 = l * s_12
y_3 = l * s_123
y_E = l * (s_1 + s_12 + s_123)


In [32]:

v_dot = 2 * R * ((y_1 + y_2 + y_3) * delta_1 + (y_2 + y_3) * delta_2 + (y_3) * delta_3) - 2 * l ** 2 * ((s_2 + s_23) * delta_2 + (s_3 + s_23) * delta_3)

qty = -delta_1*delta_1-(delta_1*delta_2*(2+c_23+2*c_3+c_2)+delta_1*delta_3*(1+c_3+c_23))/((s_1+s_12+s_123)**2+(c_1+c_12+c_123)**2)
qty_2 = delta_2*(l*c_1*delta_1*(R-l*c_1) + l**2*s_1**2*delta_1)/(l**2*s_1**2+(R-l*c_1)**2) + delta_2*((c_12*(delta_1+delta_2)+c_123*(delta_1+delta_2+delta_3))*(c_12+c_123)+(s_12*(delta_1+delta_2)+s_123*(delta_1+delta_2+delta_3))*(s_12+s_123))/((s_12+s_123)**2+(c_12+c_123)**2)
qty_2 = -qty_2
qty_3 = delta_3*(-l*(c_1*delta_1+c_12*(delta_1+delta_2))*(R-l*(c_1+c_12))-l**2*(s_1+s_12)*(s_1*delta_1+s_12*(delta_1+delta_2)))/((R-l*c_1-l*c_12)**2+l**2*(s_1+s_12)**2) - delta_3*(delta_1+delta_2+delta_3)


import numpy as np
from scipy.integrate import ode

count = 0
r = 0.5
v_dot_subs_f = -1
d_pr = np.array([0.1, 0.1, 0.1])


In [50]:
samples = np.load('samples.npy', allow_pickle=True)
print("No. of samples: ", len(samples))
for s in tqdm.tqdm(samples):
    assert s[0] < 0 or s[1] < 0 or s[2] < 0
samples[:,3] += 100
print(np.max(samples[:,3]))
#sampled_points = np.load('sampled_points.npy', allow_pickle=True)
#for s in tqdm.tqdm(sampled_points):
#    continue
#print("No. of sampled points: ", len(sampled_points))

No. of samples:  100000


100%|██████████| 100000/100000 [00:00<00:00, 2542603.40it/s]

340.4266067910662


In [34]:
samples = np.load('sample_7_pt.npy', allow_pickle=True)
print(samples.shape)
max(samples[:,6])

combos = [[0,3,5,1],
          [0,3,5,2],
          [0,3,5,4],
          [0,3,5,6],]
# combos = [[0,1,2],
#           [0,1,3],
#           [0,1,4],
#           [0,1,5],
#           [0,2,3],
#           [0,2,4],
#           [0,2,5],
#           [0,3,4],
#           [0,3,5],
#           [0,4,5],
#           [1,2,3],
#           [1,2,4],
#           [1,2,5],
#           [1,3,4],
#           [1,3,5],
#           [1,4,5],
#           [2,3,4],
#           [2,3,5],
#           [2,4,5],
#           [3,4,5]]

# combos= [[0,1],
#          [0,2],
#          [0,3],
#          [0,4],
#          [0,5],
#          [1,2],
#          [1,3],
#          [1,4],
#          [1,5],
#          [2,3],
#          [2,4],
#          [2,5],
#          [3,4],
#          [3,5],
#          [4,5]]

prev = True
for j in tqdm.tqdm(range(len(combos))):
    c = combos[j]
    qf_2 = 0
    qf_3 = 0
    for i in range(len(samples)):
        # print([1*(samples[i][c[j]]<0) for j in range(3)])
        qf = np.sum([1*(samples[i][c[j]]<0) for j in range(3)])
        # qf = np.sum([1*(samples[i][c[j]]<0) for j in range(2)])
        if  qf >= 1:
            prev = True
            if qf == 2:
                qf_2 += 1
            elif qf == 3:
                qf_3 += 1
            continue
        else:
            prev = False
            break
    if prev == True:
        print("All samples have at least one negative values for ", c, "two negatives:", qf_2, "three negatives:", qf_3)


for k in  [1,2,4,6]:
    p = 0
    p_0 = 0
    p_3 = 0
    p_5 = 0
    p_k = 0
    for i in tqdm.tqdm(range(len(samples))):
        samples[i][6] =  samples[i][0]+9*samples[i][3]+samples[i][5] + samples[i][k]
        # samples[i][6] =  samples[i][0]+samples[i][3]+samples[i][5]
        if samples[i][6] > 0:
            p += 1
            p_0 += samples[i][0]
            p_3 += samples[i][3]
            p_5 += samples[i][5]
            p_k += samples[i][k]
    print("At k:", k, "positive for:", p, "mean p_0:", p_0/p, "mean p_3:", p_3/p, "mean p_5:", p_5/p, "mean p_k:", p_k/p)
# print("positive for:", p, "mean p_0:", p_0/p, "mean p_3:", p_3/p, "mean p_5:", p_5/p)

(100000, 8)


 25%|██▌       | 1/4 [00:00<00:02,  1.35it/s]

All samples have at least one negative values for  [0, 3, 5, 1] two negatives: 21785 three negatives: 76323


 25%|██▌       | 1/4 [00:01<00:03,  1.24s/it]


KeyboardInterrupt: 

## Gradient Search

In [ ]:
cons = np.array([3.1, 9., 1.])
epochs = 100000
max_v = max(samples[:,6])
for i in tqdm.tqdm(range(epochs)):
    lr = 0.025
    pos_count = 0
    pos_count_0 = [0,0]
    pos_count_1 = [0,0]
    pos_count_2 = [0,0]
    # pos_count_3 = [0,0]

    mv = [max(samples[:,6])]*6
    for j in (range(len(samples))):
        # samples[j][6] = cons[0] * samples[j][0] + cons[1] * samples[j][3] + cons[2] * samples[j][5] + cons[3] * samples[j][4]
        samples[j][6] = cons[0] * samples[j][0] + cons[1] * samples[j][3] + cons[2] * samples[j][5]
        if samples[j][6] > 0:
            pos_count += 1
    cmv = max(samples[:,6])
    cons[0] = cons[0]-lr
    for j in range(len(samples)):
        # samples[j][6] = cons[0] * samples[j][0] + cons[1] * samples[j][3] + cons[2] * samples[j][5] + cons[3] * samples[j][4]
        samples[j][6] = cons[0] * samples[j][0] + cons[1] * samples[j][3] + cons[2] * samples[j][5]
        if samples[j][6] > 0:
            pos_count_0[0] += 1
    mv[0] = max(samples[:,6])
    cons[0] += 2*lr
    for j in range(len(samples)):
        # samples[j][6] = cons[0] * samples[j][0] + cons[1] * samples[j][3] + cons[2] * samples[j][5] + cons[3] * samples[j][4]
        samples[j][6] = cons[0] * samples[j][0] + cons[1] * samples[j][3] + cons[2] * samples[j][5]
        if samples[j][6] > 0:
            pos_count_0[1] += 1
    mv[1] = max(samples[:,6])
    cons[0] -= lr
    cons[1] -= lr
    for j in range(len(samples)):
        # samples[j][6] = cons[0] * samples[j][0] + cons[1] * samples[j][3] + cons[2] * samples[j][5]+ cons[3] * samples[j][4]
        samples[j][6] = cons[0] * samples[j][0] + cons[1] * samples[j][3] + cons[2] * samples[j][5]
        if samples[j][6] > 0:
            pos_count_1[0] += 1
    mv[2] = max(samples[:,6])
    cons[1] += 2*lr
    for j in range(len(samples)):
        # samples[j][6] = cons[0] * samples[j][0] + cons[1] * samples[j][3] + cons[2] * samples[j][5]+ cons[3] * samples[j][4]
        samples[j][6] = cons[0] * samples[j][0] + cons[1] * samples[j][3] + cons[2] * samples[j][5]

        if samples[j][6] > 0:
            pos_count_1[1] += 1
    mv[3] = max(samples[:,6])
    cons[1] -= lr
    cons[2] -= lr
    for j in range(len(samples)):
        # samples[j][6] = cons[0] * samples[j][0] + cons[1] * samples[j][3] + cons[2] * samples[j][5]+ cons[3] * samples[j][4]
        samples[j][6] = cons[0] * samples[j][0] + cons[1] * samples[j][3] + cons[2] * samples[j][5]
        if samples[j][6] > 0:
            pos_count_2[0] += 1
    mv[4] = max(samples[:,6])
    cons[2] += 2*lr
    for j in range(len(samples)):
        # samples[j][6] = cons[0] * samples[j][0] + cons[1] * samples[j][3] + cons[2] * samples[j][5]+ cons[3] * samples[j][4]
        samples[j][6] = cons[0] * samples[j][0] + cons[1] * samples[j][3] + cons[2] * samples[j][5]
        if samples[j][6] > 0:
            pos_count_2[1] += 1
    mv[5] = max(samples[:,6])
    cons[2] -= lr
    # cons[3] -= lr
    # for j in range(len(samples)):
    #     samples[j][6] = cons[0] * samples[j][0] + cons[1] * samples[j][3] + cons[2] * samples[j][5]+ cons[3] * samples[j][4]
    #     if samples[j][6] > 0:
    #         pos_count_3[0] += 1
    # cons[3] += 2*lr
    # for j in range(len(samples)):
    #     samples[j][6] = cons[0] * samples[j][0] + cons[1] * samples[j][3] + cons[2] * samples[j][5]+ cons[3] * samples[j][4]
    #     if samples[j][6] > 0:
    #         pos_count_3[1] += 1
    # cons[3] -= lr
    if min(mv) > cmv:
        print("Unable to find immediate improvement.")
        break

    # idx = np.argmin(pos_count_0+pos_count_1+pos_count_2+pos_count_3)
    idx = np.argmin(pos_count_0+pos_count_1+pos_count_2)
                    
    # if min(pos_count_0+pos_count_1+pos_count_2+pos_count_3) <= pos_count:
    if min(pos_count_0+pos_count_1+pos_count_2) <= pos_count:

        if idx == 0:
            cons[0] -= lr
        elif idx == 1:
            cons[0] += lr
        elif idx == 2:
            cons[1] -= lr
        elif idx == 3:
            cons[1] += lr
        elif idx == 4:
            cons[2] -= lr
        elif idx == 5:
            cons[2] += lr
        # elif idx == 6:
        #     cons[3] -= lr
        # elif idx == 7:
        #     cons[3] += lr

    pos_count = 0
    for j in range(len(samples)):
        # samples[j][6] = cons[0] * samples[j][0] + cons[1] * samples[j][3] + cons[2] * samples[j][5] + cons[3] * samples[j][4]
        samples[j][6] = cons[0] * samples[j][0] + cons[1] * samples[j][3] + cons[2] * samples[j][5]

        if samples[j][6] > 0:
            pos_count += 1
            
    max_v = max(samples[:,6])
    min_v = min(samples[:,6])

    if i % 10 == 0:
        print("Epoch: ", i, "Max: ", max_v, "Cons: ", cons, "Min: ", min_v, "Positive count: ", pos_count)
        

  0%|          | 0/100000 [00:00<?, ?it/s]


KeyboardInterrupt: 

## Grid Search

In [ ]:
# define c_arr taking vals from 1 to 10000
pts = 25
c_arr = np.linspace(1, pts, pts)/pts 
c_arr_1 = c_arr
c_arr_2 = c_arr
c_arr_3 = c_arr
# c_arr_1 = np.linspace(0.5-1/24, 0.5+1/24, pts) 
# c_arr_2 = np.linspace(0.4-1/24, 0.4+1/24, pts) 
# c_arr_3 = np.linspace(0.0467-1/24, 0.0467+1/24, pts) 

print("c_arr_1", c_arr_1)
print("c_arr_2", c_arr_2)
print("c_arr_3", c_arr_3)

const_arr = np.array([c_arr, c_arr, c_arr]).T
# get positive count for sample[:,3]
def get_pos_count(const_arr):
    pos_count = 0
    for j in range(len(samples)):
        samples[j][6] = const_arr[0] * samples[j][0] + const_arr[1] * samples[j][3] + const_arr[2] * samples[j][5]
        if samples[j][6] > 0:
            pos_count += 1
    return pos_count
# c = [0.5, 0.4, 0.04666666666666667]
c = [1,1,1]

c3 = None
min_pos_count = get_pos_count(c)
print("starting at:", min_pos_count)
i = 0
exit_1 = False
for i in range(len(c_arr_1)):
    c1 = c_arr_1[i]
    for j in (range(len(c_arr_2))):  
        c2 = c_arr_2[j]     
        for k in range(len(c_arr_1)):
            c3 = c_arr_3[k]
            pos_count = get_pos_count([c1, c2, c3])
            if pos_count < min_pos_count:
                min_pos_count = pos_count
                c = [c1, c2, c3]
                print("new minimum:", pos_count, "c_vals:", c, "i:", i, "j:", j, "k:", k)

            if pos_count == 0:
                exit_1 = True
        
        if exit_1:
            break
    if exit_1:
        print("Found 0 positive at:", [c1,c2,c3])
        break
    

c_arr_1 [0.04 0.08 0.12 0.16 0.2  0.24 0.28 0.32 0.36 0.4  0.44 0.48 0.52 0.56
 0.6  0.64 0.68 0.72 0.76 0.8  0.84 0.88 0.92 0.96 1.  ]
c_arr_2 [0.04 0.08 0.12 0.16 0.2  0.24 0.28 0.32 0.36 0.4  0.44 0.48 0.52 0.56
 0.6  0.64 0.68 0.72 0.76 0.8  0.84 0.88 0.92 0.96 1.  ]
c_arr_3 [0.04 0.08 0.12 0.16 0.2  0.24 0.28 0.32 0.36 0.4  0.44 0.48 0.52 0.56
 0.6  0.64 0.68 0.72 0.76 0.8  0.84 0.88 0.92 0.96 1.  ]
starting at: 1019
new minimum: 332 c_vals: [0.04, 0.08, 0.04] i: 0 j: 1 k: 0
new minimum: 291 c_vals: [0.04, 0.12, 0.04] i: 0 j: 2 k: 0
new minimum: 278 c_vals: [0.04, 0.16, 0.04] i: 0 j: 3 k: 0
new minimum: 269 c_vals: [0.04, 0.2, 0.04] i: 0 j: 4 k: 0
new minimum: 256 c_vals: [0.04, 0.24, 0.04] i: 0 j: 5 k: 0
new minimum: 247 c_vals: [0.04, 0.28, 0.04] i: 0 j: 6 k: 0
new minimum: 239 c_vals: [0.08, 0.24, 0.04] i: 1 j: 5 k: 0
new minimum: 229 c_vals: [0.08, 0.28, 0.04] i: 1 j: 6 k: 0
new minimum: 221 c_vals: [0.08, 0.32, 0.04] i: 1 j: 7 k: 0
new minimum: 220 c_vals: [0.12, 0.32, 0.04] 

Traceback (most recent call last):
  File "/home/pulkit/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_11364/3095319030.py", line 38, in <module>
    pos_count = get_pos_count([c1, c2, c3])
  File "/tmp/ipykernel_11364/3095319030.py", line -1, in get_pos_count
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pulkit/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 2142, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/pulkit/.local/lib/python3.10/site-packages/IPython/core/ultratb.py", line 1435, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/home/pulkit/.local/lib/python3.10/site-packages/IPython/core/ultratb.py", line 1326, in structured_traceback
    return VerboseTB.structured_tracebac

In [ ]:
print(c, pos_count)
get_pos_count([1,1,1])

## Neural Network Approach

In [52]:

class model(torch.nn.Module):
    def __init__(self):
        super(model, self).__init__()
        self.in_  = torch.nn.Linear(3, 1, bias=False)
        self.in_.weight.data = torch.tensor([[1.0, 1.0, 1.0]])
        # single layer network to train weights

    def forward(self, x):
        x = self.in_(x)
        return x

def train(model: torch.nn.Module, epochs, samples):
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-3)
    loss_fn = torch.nn.MSELoss()
    # train val split
    train_samples = samples[:int(len(samples) * 0.9)]
    val_samples = samples[int(len(samples) * 0.9):]
    best_loss = 10000
    
    # training loop
    for epoch in (range(epochs)):
        mean = np.mean(samples[:,3])
        variance = np.var(samples[:,3])
        running_loss = 0
        bad_train_samples = 0
        np.random.shuffle(train_samples)
        for j in tqdm.tqdm(range(len(train_samples))):
            sample = train_samples[j]
            x = torch.tensor([float(sample[0]), float(sample[1]), float(sample[2])])
            y_pred = torch.mul(model(x), torch.tensor(float(variance)))
            loss = loss_fn(y_pred, torch.mul(torch.tensor(float(min(sample[3], mean))), torch.tensor(float(variance))))
            if y_pred >=0:
                bad_train_samples += 1
            optimizer.zero_grad() 
            running_loss += loss.item()
            loss.backward()
            optimizer.step()
            for p in model.parameters():
                p.data.clamp_(0)
            # set bias to zero
        # reset sample_outs
        for k in range(len(train_samples)):
            x = torch.tensor([float(s[0]), float(s[1]), float(s[2])])
            y_pred = model(x)
            train_samples[k][3] = y_pred.item()

        # validation
        val_loss = 0
        for j in tqdm.tqdm(range(len(val_samples))):
            sample = val_samples[j]
            x = torch.tensor([float(sample[0]), float(sample[1]), float(sample[2])])
            y_pred = model(x)
            if y_pred >= 0:
                val_loss += 1
        best_loss = min(best_loss, val_loss)
        if val_loss == best_loss:
            best_model = model
        
        # outputs
        print(f"Epoch {epoch} Average Loss: {running_loss/90000} Bad Train Samples: {bad_train_samples} Bad Val Samples: {val_loss} Mean: {mean} Variance: {variance}")
        if val_loss == 0 and bad_train_samples == 0:
            print("No bad samples found, training terminated.")
            break
    return best_model
# create model
model = model()


# train model
best_model = train(model, 10000, samples)


  0%|          | 0/90000 [00:00<?, ?it/s]/home/pulkit/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████| 10000/10000 [00:00<00:00, 37509.26it/s]


Epoch 0 Average Loss: 229992309.65852752 Bad Train Samples: 204 Bad Val Samples: 2 Mean: -12.655572562055966 Variance: 1640.0419180191923


100%|██████████| 90000/90000 [00:28<00:00, 3201.76it/s]


In [ ]:
print("Running experiment with model:")
model = best_model
for p in model.parameters():
    if p.requires_grad:
         print(p.name, p.data)
    # print bias
    else:
        print(p.name, p.data)

Running experiment with model:
None tensor([[0.3333, 0.3333, 0.3333]])


In [ ]:
v_dot_subs_f = -1
count = 0
# while count < 100000 and v_dot_subs_f < 0:
for i in tqdm.tqdm(range(100000)):
    phi_1_subs = -np.pi + 2 * np.pi * np.random.rand()
    phi_2_subs = -r * np.pi + 2 * r * np.pi * np.random.rand()
    phi_3_subs = -r * np.pi + 2 * r * np.pi * np.random.rand()
    R_subs = 0.1 + 2.9 * np.random.rand()
    l_subs = 1
    delta_1_subs = float(delta_1.subs({phi_1: phi_1_subs, phi_2: phi_2_subs, phi_3: phi_3_subs, l: l_subs, R: R_subs, alpha: 0}))
    delta_2_subs = float(delta_2.subs({phi_1: phi_1_subs, phi_2: phi_2_subs, phi_3: phi_3_subs, l: l_subs, R: R_subs, alpha: 0}))
    delta_3_subs = float(delta_3.subs({phi_1: phi_1_subs, phi_2: phi_2_subs, phi_3: phi_3_subs, l: l_subs, R: R_subs, alpha: 0}))
    l_ = 1
    k1 = R_subs
    k2 = 0
    f = [phi_1_subs, phi_2_subs, phi_3_subs]
    x_0 = [l_ * np.cos(f[0]), l_ * np.sin(f[0]), l_ * np.cos(f[0]) + l_ * np.cos(f[0] + f[1]), l_ * np.sin(f[0]) + l_ * np.sin(f[0] + f[1]), l_ * np.cos(f[0]) + l_ * np.cos(f[0] + f[1]) + l_ * np.cos(f[0] + f[1] + f[2]), l_ * np.sin(f[0]) + l_ * np.sin(f[0] + f[1]) + l_ * np.sin(f[0] + f[1] + f[2]), f[0], f[1], f[2]]
    tspan = np.arange(0, 300.1, 0.1)
    sol = integrate.odeint(foot, x_0, tspan, args=(l_, k1, k2))
    f1_star = sol[3000, 6]
    f2_star = sol[3000, 7]
    f3_star = sol[3000, 8]
    qty_1_4 = (phi_1_subs - f1_star) * delta_1_subs
    qty_2_4 = (phi_1_subs - f1_star + phi_2_subs - f2_star) * (delta_2_subs + delta_1_subs)
    qty_3_4 = (phi_3_subs - f3_star + phi_1_subs - f1_star + phi_2_subs - f2_star) * (delta_1_subs + delta_2_subs + delta_3_subs)
    v_dot_4 = 2 * (qty_1_4 + qty_2_4 + qty_3_4)
    qty_subs_1 = float(qty.subs({phi_1: phi_1_subs, phi_2: phi_2_subs, phi_3: phi_3_subs, l: l_subs, R: R_subs, alpha: 0}))
    qty_subs_2 = float(qty_2.subs({phi_1: phi_1_subs, phi_2: phi_2_subs, phi_3: phi_3_subs, l: l_subs, R: R_subs, alpha: 0}))
    qty_subs_3 = float(qty_3.subs({phi_1: phi_1_subs, phi_2: phi_2_subs, phi_3: phi_3_subs, l: l_subs, R: R_subs, alpha: 0}))
    v_dot_3 = 2 * (qty_subs_1 + qty_subs_2 + qty_subs_3)
    v_dot_subs = float(v_dot.subs({phi_1: phi_1_subs, phi_2: phi_2_subs, phi_3: phi_3_subs, l: l_subs, R: R_subs, alpha: 0}))
    v_dot_subs_f = model(torch.tensor([float(v_dot_subs), float(v_dot_3), float(v_dot_4)]))
    if v_dot_subs_f > 0:
        break
if count == 100000:
    print("Sim completed successfully")
else:
    print("Positive v_dot found, value", v_dot_subs_f)

  0%|          | 4/100000 [00:00<5:17:13,  5.25it/s]

Positive v_dot found, value tensor([10.0718], grad_fn=<SqueezeBackward4>)
